In [1]:
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import gradio as gr

In [2]:
# Initialize components
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
llm = Ollama(
    model="llama2",  
    base_url="http://localhost:11434",  
    temperature=0.3
)

C:\Users\Nidhi.Chakravarthy\AppData\Local\Temp\ipykernel_23760\4032639944.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\Nidhi.Chakravarthy\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\Nidhi.Chakravarthy\AppData\Local\Temp\ipykernel_23760\4032639944.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated 

In [3]:
# Load ESG base documents
def load_esg_base():
    loader = PyPDFLoader("C:/Users/Nidhi.Chakravarthy/Documents/Financial-Document-parser-using-RAG/esg_regulations.pdf")
    pages = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return splitter.split_documents(pages)

vector_store = FAISS.from_documents(load_esg_base(), embeddings)

In [4]:
def analyze_content(query, file):
    try:
        if file:
            loader = PyPDFLoader(file.name)
            user_docs = loader.load()
            splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            user_chunks = splitter.split_documents(user_docs)
            
            user_store = FAISS.from_documents(user_chunks, embeddings)
            user_store.merge_from(vector_store)
            retriever = user_store.as_retriever()
        else:
            retriever = vector_store.as_retriever()

        prompt = ChatPromptTemplate.from_template(
            """Analyze this ESG document for compliance risks:
            {context}
            
            Query: {input}
            
            Format findings as:
            - [RISK LEVEL] [SECTION]: [DESCRIPTION]"""
        )

        chain = (
            {"context": retriever, "input": RunnablePassthrough()}
            | prompt
            | llm
        )

        return chain.invoke(query)
    
    except Exception as e:
        return f"Error: {str(e)}"

In [5]:
# Gradio interface
with gr.Blocks(title="ESG Compliance Analyzer") as app:
    gr.Markdown("## ESG Document Analyzer with Local LLM")
    
    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="Upload Document (PDF)")
            query_input = gr.Textbox(label="Your Compliance Question")
            submit_btn = gr.Button("Analyze")
        
        with gr.Column():
            output = gr.Textbox(label="Analysis Results", interactive=False)

    submit_btn.click(
        fn=analyze_content,
        inputs=[query_input, file_input],
        outputs=output
    )

if __name__ == "__main__":
    app.launch(server_name="0.0.0.0", share=False)

Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.
